In [1]:
pip install yfinance

     |████████████████████████████████| 5.5MB 5.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/37/fa/73/d492849e319038eb4d986f5152e4b19ffb1bc0639da84d2677
Successfully built yfinance multitasking
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install edgar

     |████████████████████████████████| 71kB 5.2MB/s eta 0:00:011
     |████████████████████████████████| 51kB 11.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein
Note: you may need to restart the kernel to use updated packages.


In [3]:
#import Libraries
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta
import datetime
import calendar
import yfinance as yf
import datetime as dt
import my_project_library as mylib
%load_ext autoreload
%autoreload 2



In [4]:

!conda install -c anaconda lxml --yes
!pip install lxml
print('LXML parser installed')
import lxml.html

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.7.22  |                0         132 KB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    libxml2-2.9.10             |       he19cac6_1         1.3 MB  anaconda
    libxslt-1.1.34             |       hc22bd24_0         573 KB  anaconda
    lxml-4.5.2                 |   py36hefd8a0e_0         1.4 MB  anaconda
    openssl-1.1.1h             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         7.3 MB

The following NEW packages will be INSTALLED:

  libxslt            anaconda/linux-64::libxslt-1.1.34-hc22bd24_0
  lxml           

In [5]:
#pull in NLP_dataset
'''bring in the spx nlp datset from the main program and repickle the data to save down that work'''

data = pickle.load( open( "spx_filings_nlp.p", "rb" ) )
print(data.head())

  Symbol     Company  GICS Sector         GICS Sub Industry         CIK  Year  \
0    MMM  3M Company  Industrials  Industrial Conglomerates  0000066740  2019   
1    MMM  3M Company  Industrials  Industrial Conglomerates  0000066740  2018   
2    MMM  3M Company  Industrials  Industrial Conglomerates  0000066740  2017   
3    MMM  3M Company  Industrials  Industrial Conglomerates  0000066740  2016   
4    MMM  3M Company  Industrials  Industrial Conglomerates  0000066740  2015   

  Filing_Date   Cos_Sim  Word_chg  
0  2020-02-06  0.932309 -0.101716  
1  2019-02-07  0.990988 -0.006023  
2  2018-02-08  0.991195  0.033727  
3  2017-02-09  0.993092 -0.023340  
4  2016-02-11  0.985685 -0.073579  


In [6]:
#preprocess our data to agree with Yahoo finance
data.drop('CIK', axis=1, inplace=True)
data.rename(columns = {'Symbol':'Ticker'}, inplace = True)
data.rename(columns = {'Year':'Report_Period'}, inplace = True)

In [7]:
#first convert our lxml date object to datetime format
data['Filing_Date'] = pd.to_datetime(data['Filing_Date'].astype(str), format='%Y-%m-%d', errors="coerce")
data.dtypes
data['Filing_Date'] = pd.to_datetime(data['Filing_Date'], format='%Y-%m-%d %H:%M:%S').dt.to_period('d')

#converting today's date to a periodic version for comparison
today = dt.date.today()
today = pd.to_datetime(today)
today = today.to_period(freq ='d')

filing_returns = len(data['Filing_Date'])
# print(filing_returns)

price_chg_dict =  {'Ticker':[],'Date':[],'Adj_Close':[] }


In [8]:
data.shape

(2185, 8)

In [8]:

# Define the weekday mnemonics to match the date.weekday function
(MON, TUE, WED, THU, FRI, SAT, SUN) = range(7)
# Define default weekends, but allow this to be overridden at the function level
default_weekends=(SAT,SUN)
def workday(start, days, holidays=[], weekends=default_weekends):
    """
    Repurposed to suit my needs for date processing business days from a periodic format
    Then casted back into a periodic form to process with Yahoo Finance returns
    -- started from the code of Casey Webster at
    http://groups.google.com/group/comp.lang.python/browse_thread/thread/ddd39a02644540b7
    """

    sd= start.strftime("%Y-%m-%d")
    start_date = datetime.datetime.strptime(sd,'%Y-%m-%d')

    if days == 0:
        return start_date;
    if days>0 and start_date.weekday() in weekends: #
      while start_date.weekday() in weekends:
          start_date -= timedelta(days=1)
    elif days < 0:
      while start_date.weekday() in weekends:
          start_date += timedelta(days=1)
    full_weeks, extra_days = divmod(days,7 - len(weekends))
    new_date = start_date + timedelta(weeks=full_weeks)
    for i in range(extra_days):
        new_date += timedelta(days=1)
        while new_date.weekday() in weekends:
            new_date += timedelta(days =1)
    # to account for days=0 case
    while new_date.weekday() in weekends:
        new_date += timedelta(days=1)
    # avoid this if no holidays
    if holidays:
        delta = timedelta(days=1 * cmp(days,0))
        # skip holidays that fall on weekends
        holidays =  [x for x in holidays if x.weekday() not in weekends ]
        holidays =  [x for x in holidays if x != start_date ]
        for d in sorted(holidays, reverse = (days < 0)):
            # if d in between start and current push it out one working day
            if _in_between(start_date, new_date, d):
                new_date += delta
                while new_date.weekday() in weekends:
                    new_date += delta
    new_date=new_date.strftime("%Y-%m-%d")
    new_date = pd.to_datetime(new_date, format='%Y-%m-%d').to_period(freq ='d')
    return new_date


In [9]:
def get_px_chgs(today,ticker_select, start, end):
    """
    created to retrieve prices from Yahoo finance function.
    needs the start (filing date) and end date(assigned in the main program at 200_days from filing)
    despite only looking at 90 days, having the flexibility to extend to 6mos if data warrants
    calculating an average price given the high,low, and close.  ideally want the VWAP - calc in future update
    """
    print(ticker_select) #maintain status of ticker processing in case of error in data
    ticker_prices = yf.download(ticker_select, start=start, end=end, auto_adjust=False) #run for each filing on each ticker -  4 filings per ticker
    # ticker_prices['AvgPx'] = ticker_prices.iloc[:,1:4].sum(axis=1) / 3  #average of high, low, close - calc vwap?

    ticker_prices.reset_index(inplace=True)
    adj_close, date = [], []
    for y in range(len(ticker_prices)):
        adj_close.append(ticker_prices['Adj Close'][y])
        date.append(ticker_prices['Date'][y])

    return ticker_select,date,adj_close


In [10]:
for x in range(filing_returns):
    start = data['Filing_Date'][x]  #begin pricing period from the filing date
  #checking the date is not out of bounds
    if today < start+timedelta(days=252):
        date_diff_days = today - start
        date_diff_days = date_diff_days/np.timedelta64(1,'D') #convert to day periodic
    else:
        date_diff_days=252 #trading days in the year

    future_ret_date = start+timedelta(days=date_diff_days)
    date_30 = workday(start,30)
    date_90 = workday(start,90)
    date_180 = workday(start,180)
    end = future_ret_date
    ticker_select = data['Ticker'][x]

  #run function to retrivew price changes from each filing date
  #this pipeline retrieves avg prices from each respective filing date
    price_chgs = get_px_chgs(today,ticker_select,start,end)
  # update price changes
    price_chg_dict['Ticker'].append(price_chgs[0]),
    price_chg_dict['Date'].append(price_chgs[1]),
    price_chg_dict['Adj_Close'].append(price_chgs[2]
    )


MMM
[*********************100%***********************]  1 of 1 completed
MMM
[*********************100%***********************]  1 of 1 completed
MMM
[*********************100%***********************]  1 of 1 completed
MMM
[*********************100%***********************]  1 of 1 completed
MMM
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%*****************

### Rebuild the dataframe that incorporates all prices

In [11]:
#reuild the dataframe
price_chg_df = pd.DataFrame(price_chg_dict)
result = pd.merge(data, price_chg_df, right_index=True, left_index=True)
#since the date and price columns will have nested lists, we need to use explode here to break it out
data_prices = result.apply(pd.Series.explode).reset_index()
data_prices.drop(['index', 'Ticker_y'], axis = 1, inplace=True)
data_prices.rename(columns = {'Ticker_x':'Ticker'}, inplace = True)
print(data_prices.head())


  Ticker     Company  GICS Sector         GICS Sub Industry Report_Period  \
0    MMM  3M Company  Industrials  Industrial Conglomerates          2019   
1    MMM  3M Company  Industrials  Industrial Conglomerates          2019   
2    MMM  3M Company  Industrials  Industrial Conglomerates          2019   
3    MMM  3M Company  Industrials  Industrial Conglomerates          2019   
4    MMM  3M Company  Industrials  Industrial Conglomerates          2019   

  Filing_Date   Cos_Sim  Word_chg       Date Adj_Close  
0  2020-02-06  0.932309 -0.101716 2020-02-06   158.401  
1  2020-02-06  0.932309 -0.101716 2020-02-07   155.668  
2  2020-02-06  0.932309 -0.101716 2020-02-10   155.736  
3  2020-02-06  0.932309 -0.101716 2020-02-11   157.973  
4  2020-02-06  0.932309 -0.101716 2020-02-12   159.286  


In [12]:
with open('filings_returns_nlp.p', 'wb') as f:
     pickle.dump(data_prices, f)

In [13]:
data_prices.head()

,Ticker,Company,GICS Sector,GICS Sub Industry,Report_Period,Filing_Date,Cos_Sim,Word_chg,Date,Adj_Close
0,MMM,3M Company,Industrials,Industrial Conglomerates,2019,2020-02-06,0.932309,-0.101716,2020-02-06,158.401
1,MMM,3M Company,Industrials,Industrial Conglomerates,2019,2020-02-06,0.932309,-0.101716,2020-02-07,155.668
2,MMM,3M Company,Industrials,Industrial Conglomerates,2019,2020-02-06,0.932309,-0.101716,2020-02-10,155.736
3,MMM,3M Company,Industrials,Industrial Conglomerates,2019,2020-02-06,0.932309,-0.101716,2020-02-11,157.973
4,MMM,3M Company,Industrials,Industrial Conglomerates,2019,2020-02-06,0.932309,-0.101716,2020-02-12,159.286


In [45]:
new_frame.shape

(13986, 10)

In [15]:
#reload the pickled data
data = pickle.load( open( "filings_returns_nlp.p", "rb" ) )
print(data.head())

  Ticker     Company  GICS Sector         GICS Sub Industry  Year Filing_Date  \
0    MMM  3M Company  Industrials  Industrial Conglomerates  2019  2020-02-06   
1    MMM  3M Company  Industrials  Industrial Conglomerates  2019  2020-02-06   
2    MMM  3M Company  Industrials  Industrial Conglomerates  2019  2020-02-06   
3    MMM  3M Company  Industrials  Industrial Conglomerates  2019  2020-02-06   
4    MMM  3M Company  Industrials  Industrial Conglomerates  2019  2020-02-06   

    Cos_Sim  Word_chg       Date Adj_Close  
0  0.932309 -0.101716 2020-02-06   158.401  
1  0.932309 -0.101716 2020-02-07   155.668  
2  0.932309 -0.101716 2020-02-10   155.736  
3  0.932309 -0.101716 2020-02-11   157.973  
4  0.932309 -0.101716 2020-02-12   159.286  
